In [2]:
import os
import pandas as pd

train_data_path ="~/aiffel/240624-dlthon/data/train.csv"
train_data = pd.read_csv(train_data_path)
train_data.head()

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [3]:
print(train_data["conversation"][0])
print(len(train_data))

지금 너 스스로를 죽여달라고 애원하는 것인가?
 아닙니다. 죄송합니다.
 죽을 거면 혼자 죽지 우리까지 사건에 휘말리게 해? 진짜 죽여버리고 싶게.
 정말 잘못했습니다.
 너가 선택해. 너가 죽을래 네 가족을 죽여줄까.
 죄송합니다. 정말 잘못했습니다.
 너에게는 선택권이 없어. 선택 못한다면 너와 네 가족까지 모조리 죽여버릴거야.
 선택 못하겠습니다. 한번만 도와주세요.
 그냥 다 죽여버려야겠군. 이의 없지?
 제발 도와주세요.
3950


1. **데이터 EDA와 데이터 전처리가 적절하게 이뤄졌는가?**
    * 결측치 제거
    * 중복데이터 제거
    * 불용어 제거 - 생성 모델이 아니기때문에 불용어를 제거해 중요한 단어만 넣는것이 중요하다고 생각
    * 특수문자제거 또는 space bar로 변환
    * 영어가 섞여있을 경우 소문자로 변환
    * 숫자 처리는 어떻게 해야하는가? 숫자가 섞인 욕을 하는 경우가 있을 수 잇음. -> 숫자 그대로를 유지하는게 나을지 아니면 숫자를 의미를 가진 단어로 변환을 할지.
    * 형태소 분석기를 사용하여 단어사전의 규모를 줄이는게 나을 수도 있지 않을까함. 그럴려면 맞춤법 검사기를 돌려야하나 생각이 들긴 함.
    * 토큰화와 패딩.
    * one hot encoding
    * 데이터를 분할
2. **Task에 알맞게 적절한 모델을 찾아보고 선정했는가?**
    * 임베딩 레이어를 통해 고정된 크기의 벡터로 변환
    * RNN: LSTM, GRU
        * 대화의 흐름을 이해하고 감정을 예측하는 데 효과적
        * LSTM과 GRU는 긴 시퀀스에서도 정보를 기억하고 사용하여 문장의 전후 맥락을 이해할 수 있음. 따라서 대화의 맥락을 이해하고 위협성을 판단하는 데 유리
        * * loss = Sparse Categorical Crossentropy (다중 클래스 분류 문제에서 주로 사용되기 때문)
    * 1D CNN
        * : CNN은 텍스트 데이터에서 지역 패턴(예: 특정 단어 조합이나 구문)을 감지하는 데 뛰어납니다. 이는 특정 위협적인 표현을 감지하는 데 유리
        * loss='categorical_crossentropy'
    * BERT 한국어 모델
        * BERT는 문장의 양방향 문맥을 이해
        * BERT는 주로 텍스트 분류, 문장 유사성 평가 등에 사용
        * loss='sparse_categorical_crossentropy'
    
3. **성능향상을 위해 논리적으로 접근했는가?**
4. **결과 도출을 위해 여러가지 시도를 진행했는가?**
5. **도출된 결론에 충분한 설득력이 있는가?**
6. **적절한 metric을 설정하고 그 사용 근거 및 결과를 분석하였는가?**
    * 정확도(Accuracy): 전체 데이터 중 맞게 분류된 데이터의 비율.
    * 정밀도(Precision): 실제로 위협이 아닌 대화를 위협으로 잘못 예측한 경우를 줄이기 위한 지표.
    * 재현율(Recall): 실제 위협 대화를 위협으로 잘 예측하는 비율.
    * Confusion Matrix: 모델이 예측한 클래스와 실제 클래스 간의 혼동을 시각적으로 표현하여 어떤 클래스에서 오류가 발생하는지 확인
7. **발표가 매끄럽게 진행되었고 발표시간을 준수하였는지?**

### 결측치 제거

In [4]:
print(train_data.isnull().sum())

idx             0
class           0
conversation    0
dtype: int64


In [5]:
print(train_data.isnull().sum()['class'])

0


### 중복제거

In [6]:
print(len(train_data))

3950


In [7]:
print('conversation열에서 중복을 배제한 유일한 샘플의 수 :', train_data['conversation'].nunique())

conversation열에서 중복을 배제한 유일한 샘플의 수 : 3846


In [8]:
print(train_data['conversation'].value_counts())

저기요. 잠깐만요\n매일 왜이러세요. 전 할말 없어요\n그 쪽이 좋아서요\n전 관심 없어요. 오지 마세요\n한 번만 만나주세요.\n제발 이러지 마세요. 전 남자친구 있어요\n결혼한 건 아니잖아요. 딱 한번만 데이트해요\n내가 왜 그쪽이랑 만나요?\n그 쪽이 내 스타일이에요\n난 아니거든요.                                                                                                                                                        2
야 석탄?\n 내가 왜 석탄이야?\n 얼굴이 까마니까 석탄이지.\n 그만해.\n 너 라이터 불붙이면 아주 잘 타겠다.\n 가 비켜.\n 아주 콧구멍 커지니까 이제 연탄이네.\n 비키라고.\n 야 연탄?\n 제발 좀 그만해. 나 정말 힘들어.                                                                                                                                                                                    2
진용아 내 내 가방 돌려줘\n아하하 진용아? 가방 돌려줄까?\n노 놀리지마\n노 놀리지마\n하아.\n한숨 쉰 거야?\n아 아 아아니야.\n아 아 아 아니야? 하하하하\n아아.\n왜? 기분 나빠?                                                                                                                                                                                                              2
김대리. 이번 주간 회의 발표 준비좀 해줘.\n저 말입니까.?\n응. 너가 해줘\n과장님 저번

In [9]:
print(train_data[train_data['conversation'] == "저기요. 잠깐만요\n매일 왜이러세요. 전 할말 없어요\n그 쪽이 좋아서요\n전 관심 없어요. 오지 마세요\n한 번만 만나주세요.\n제발 이러지 마세요. 전 남자친구 있어요\n결혼한 건 아니잖아요. 딱 한번만 데이트해요\n내가 왜 그쪽이랑 만나요?\n그 쪽이 내 스타일이에요\n난 아니거든요."])

     idx      class                                       conversation
262  262  기타 괴롭힘 대화  저기요. 잠깐만요\n매일 왜이러세요. 전 할말 없어요\n그 쪽이 좋아서요\n전 관심...
932  932  기타 괴롭힘 대화  저기요. 잠깐만요\n매일 왜이러세요. 전 할말 없어요\n그 쪽이 좋아서요\n전 관심...


In [10]:
print(train_data[train_data['conversation'] == '야 석탄?\n 내가 왜 석탄이야?\n 얼굴이 까마니까 석탄이지.\n 그만해.\n 너 라이터 불붙이면 아주 잘 타겠다.\n 가 비켜.\n 아주 콧구멍 커지니까 이제 연탄이네.\n 비키라고.\n 야 연탄?\n 제발 좀 그만해. 나 정말 힘들어.'])

       idx      class                                       conversation
782    782  기타 괴롭힘 대화  야 석탄?\n 내가 왜 석탄이야?\n 얼굴이 까마니까 석탄이지.\n 그만해.\n 너...
2656  2656  기타 괴롭힘 대화  야 석탄?\n 내가 왜 석탄이야?\n 얼굴이 까마니까 석탄이지.\n 그만해.\n 너...


In [11]:
# inplace=True 를 설정하면 DataFrame 타입 값을 return 하지 않고 data 내부를 직접적으로 바꿉니다
train_data.drop_duplicates(subset = ['conversation'], inplace=True)
print('전체 샘플수 :', (len(train_data)))

전체 샘플수 : 3846


In [13]:
#불용어
stopwords = ['것','저','그','의', '가', '이', '은', '는', '을', '를', '에', '와', '과','도','로', '하다', '다','이다', '하다', '되다', '뿐', '한테', '으로', '']

In [15]:
from konlpy.tag import Okt

# Okt 형태소 분석기 객체 생성
okt = Okt()


# def lemmatize_text(text):
#     # 텍스트를 형태소와 품사로 분석
#     morphs = okt.pos(text, stem=True)
#     # 원형이 있는 형태소만 선택
#     lemmatized = [word for word, pos in morphs]
#     return ' '.join(lemmatized)

# # 예시 텍스트
# sample_text = "저는 오늘 그 사람을 만났습니다. 그리고 그 사람은 매우 친절했습니다."

# # 원형 변환 적용
# lemmatized_text = lemmatize_text(sample_text)

# print("Original Text:", sample_text)
# print("Lemmatized Text:", lemmatized_text)


def morph_analyze(text):
    # 텍스트에서 형태소 분석
    morphs = okt.morphs(text)
    return morphs

In [16]:
# import requests
# from bs4 import BeautifulSoup
# import json
# from urllib.request import urlopen

# def check_spelling(text):
#     # 부산대 맞춤법 검사기 URL
#     url = "http://speller.cs.pusan.ac.kr/results"
    
#     # 검사할 텍스트 데이터
#     data = {'text1': text}
    
#     # 요청 보내기
#     response = requests.post(url, data=data)
    
#     # BeautifulSoup을 사용하여 HTML 파싱
#     soup = BeautifulSoup(response.text, 'html.parser')
    
#     # 맞춤법 교정 결과 가져오기
#     result = soup.find('table', {'class': 'table'}).find_all('td')
    
#     corrected_text = text
#     for idx in range(0, len(result), 3):
#         original = result[idx].text.strip()
#         suggestion = result[idx + 1].text.strip()
#         if suggestion != '맞음':
#             corrected_text = corrected_text.replace(original, suggestion)
    
#     return corrected_text

# # 예시 텍스트
# sample_text = "저는 오늘 그 사람을 만났읍니다. 그리고 그 사람은 매우 친절했읍니다."

# # 맞춤법 검사 적용
# corrected_text = check_spelling(sample_text)

# print("Original Text:", sample_text)
# print("Corrected Text:", corrected_text)

In [19]:
import re

def preprocess_sentence(sentence, remove_stopwords=True):
    # 맞춤법 검사
    
    # 특정 단어는 @~@되어있기 때문에 처리필요.
    
    
    #형태소 분석
    sentence = morph_analyze(sentence)
    
#     # 영어라면 소문자로 변환
#     sentence = sentence.lower() # 텍스트 소문자화

#     # 단어와 구두점 사이의 거리를 만들기
#     sentence = re.sub(r"([?.!])", r" \1", sentence)

#     # \n을 'ENTER'으로 변환
#     sentence = re.sub(r'[\n]', " ENTER ", sentence)

#     # 특수문자 제거 (문장내의 특수 문자제거)
#     sentence = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z?.!\s]", "", sentence)
    
    
    # 불용어 제거 (Text)     
    sentence = ' '.join(word for word in sentence if  word not in stopwords)

    return sentence

In [20]:
temp = train_data['conversation'].map(lambda x: preprocess_sentence(x))

In [24]:
train_data['conversation'][10:30]

10    네 뭐 드릴까요?\n말보르 레드 하나.\n네. 여기요!\n여기 사장은 없어?\n네 ...
11    이대리 오늘 도대체 뭘한거야?\n창고정리도 하고 바닥도 청소했습니다\n이게 한거니?...
12    네네 무슨 일 때문에 전화주셨나요?\n우리 애가 지우개 하나 훔친거 가지고 애들 앞...
13    형! 형! 돈좀있어?\n저 저요.?\n여기 형말고 누가 있어? 형 얼마 있어?\n저...
14    김사원\n대리님 왜 그러세요?\n이거 오늘까지 제출해야 하는데 좀 해줄래?\n네? ...
15    야 너 휴대폰좀 빌려줘봐\n안돼. 지난달에도 휴대폰 요금이 너무 많이 나와서 엄마한...
16    잠깐만 거기 학생 \n예? 저 부르셨너요?\n그럼 누구겠어 \n왜부르시죠\n내가 지...
17    얼마 가져왔어?\n십만원 정도.\n야 이걸로 새 폰을 어떻게 사\n이게 최선이었어....
18    야 너이마빡에 빛이난다\n어?\n야 한대만 때려보자 퍽\n아 흑 왜그래 \n아팠어?...
19    나씨 정수기 물탱크 좀 갈아주시겠어요?\n싫습니다. 물 저만 마시는 것도 아닌데 매...
20    너가 민수라는 남자 제자니?\n네 맞는데요 왜 그러시죠? 혹시 저희 선생님께 무슨 ...
21    너 저번에 친구 지갑에서 돈 훔쳤잖아.\n그말을 왜 지금 꺼내.\n야 다 불어버리기...
22    어유 파리꼬여 어디서 계란썩는 냄새가 나냐 설마 사람한태 나는 냄새는 아닐테고\n너...
23    나좀 제발 그만 따라다녀.부탁이야\n너 만나려면 널 찾아가야될거아냐\n우리 헤어졌자...
24    야. 거지!\n너네집 저기 판자촌이라면서?\n.\n판자촌이 뭐야?\n왜 저 위에 다...
25    여보 생활비좀 주세요\n아니 이놈의 마누라는 허구한날 입만 열면 돈타령이야\n애들 ...
26    조용히 살라고 했는데 이새끼가깔짝깔짝건드네?\n그러지마.그건 정말 너네가 잘못한거잖...
27    돈을 빌려갔으면 갚아야지\n죄송합니다 시간을 조금만 더 주십시오\n내가 왜?

In [23]:
temp[10:30]

10    네 뭐 드릴 까요 ? \n 말 보르 레드 하나 . \n 네 . 여기 요 ! \n 여...
11    대리 오늘 도대체 뭘 한거야 ? \n 창고 정리 하고 바닥 청소 했습니다 \n 게 ...
12    네네 무슨 일 때문 전화 주셨나요 ? \n 우리 애가 지우개 하나 훔친거 가지 고 ...
13    형 ! 형 ! 돈 좀있어 ? \n 요 .? \n 여기 형 말고 누가 있어 ? 형 얼...
14    김사원 \n 대리 님 왜 그러세요 ? \n 거 오늘 까지 제출 해야 하는데 좀 해줄...
15    야 너 휴대폰 좀 빌려줘 봐 \n 안 돼 . 지난달 에도 휴대폰 요금 너무 많이 나...
16    잠깐 만 거기 학생 예 ? 부르셨 너 요 ? \n 그럼 누구 겠어 왜 부르시죠 \n...
17    얼마 가져왔어 ? \n 십만원 정도 . \n 야 걸 새 폰 어떻게 사 \n 게 최선...
18    야 너 이마 빡 빛 난 \n 어 ? \n 야 한 대만 때려 보자 퍽 \n 아 흑 왜...
19    나씨 정수기 물탱크 좀 갈아 주시겠어요 ? \n 싫습니다 . 물 저만 마시는 아닌데...
20    너 민수 라는 남자 제자 니 ? \n 네 맞는데요 왜 러시 죠 ? 혹시 저희 선생님...
21    너 저번 친구 지갑 에서 돈 훔쳤잖아 . \n 말 왜 지금 꺼내 . \n 야 불어 ...
22    어유 파리 꼬여 어디서 계란 썩는 냄새 나냐 설마 사람 한 태 나 냄새 아닐테고 \...
23    나좀 제발 그만 따라다녀 . 부탁 이야 \n 너 만나려면 널 찾아가야 될거아냐 \n...
24    야 . 거지 ! \n 너 네 집 저기 판자촌 이라 면서 ? \n . \n 판자촌 뭐...
25    여보 생활비 좀 주세요 \n 아니 놈 마누라 허구 한 날 입 만 열면 돈 타령 이야...
26    조용히 살라 고 했는데 새끼 깔짝깔짝 건드 네 ? \n 그러지마 . 그건 정말 너 ...
27    돈 빌려 갔으면 갚아야지 \n 죄송합니다 시간 조금 만 더 주 십 시오 \n